In [62]:
import sys
import os
import mmh3
import numpy as np
import itertools
import collections
import pandas as pd
import more_itertools
from itertools import tee
import re
#################### Utilities ######################
#hashes a list of strings
def listhash(l,seed):
	val = 0
	for e in l:
		val = val ^ mmh3.hash(e, seed)
	return val 

################### Similarity ######################

docs = {} #dictionary mapping document id to document contents

# datafolder = os.path.join("C:\\Users\\hasee\\Documents\\comptools\\week5\\DataWeek5", "ats_corpus_small")   # change to ats_corpus for large data set

# for file in os.listdir(datafolder):
#     filepath = os.path.join(datafolder, file)
#     f = open(filepath, 'r')
#     docs[file] = f.read()
#     print("read document " + file)
#     f.close()

data = pd.read_csv(r"C:\Users\hasee\Documents\comptools\DTU_CTDS\data\articles1.csv")

#%%

data1 = data[:1000]
key = data1["Unnamed: 0"]
data2 = data1["content"]
docs = data2.to_dict()
# docs = data2.tolist()
# g_docs = data2.tolist()
docs = list(zip(data2.tolist())) ## convert to tuple for starmap to work basically getting ['dfdf', 'dddd', 'sss'] => [('dfdf'), ('dddd'), ('sss')]
# print(docs[2])

In [63]:
''' Extension of standard pairwise function to 3-pairwise from the py standard lib'''
def three_pairwise(iterable):
    # pairwise('ABCDEFG') --> AB BC CD DE EF FG
    # three_pairwise('ABCDEFG') --> ABC BCD CDE DEF EFG FG
    a, b, c = tee(iterable, 3) ## Create three iterators
    next(b, None) ## Advance the second
    next(c, None) ## Advance the third
    next(c, None) ## Advance the third once again. This ensures the third iterator starts at the third element and that we can create the 3-tuple
    return zip(a, b, c) ### Zip everything (1 elem, 2 elem, and 3 elem) concurrently.

def hash_docs(doc):
    # three_tup=zip
    lst_shingles_h=[]
    # for doc in docs:
    three_tup=three_pairwise(doc.split(" "))
    seed=0
    return [listhash(shingle, seed) for shingle in three_tup]  # 

''' Returns document as a list of hashes'''
'''
Creates shingles of size q, removing shingles of size < q. Removes duplicates and hashes the result.
'''
def hashed_lst_shingles(my_docs):
        
    # seed=0
    # doc = doc.split(" ")
    # lst_shingles_h = [listhash(doc[i:i+q], seed) for i in range(0, len(doc), q-2) if len(doc[i:i+q])==q] # create list of shingles of length q
    # return lst_shingles_h

    '''More efficient implementation'''

    # https://stackoverflow.com/questions/3862010/is-there-a-generator-version-of-string-split-in-python
    # def split_iter(string):
    #     return (x.group(0) for x in re.finditer(r"[A-Za-z']+", string)) # maybe remove list() made it 10x faster?

    hashed_docs = itertools.starmap(hash_docs, my_docs)
    return hashed_docs, len(my_docs)

# doc = "You and me, we made a vow. For better or for worse. I can't believe you let me down"
# lst_shnigles = hashed_lst_shingles(3, doc)
# print(lst_shnigles)
# [['You', 'and', 'me,'], ['and', 'me,', 'we'], ['me,', 'we', 'made'], ['we', 'made', 'a'], ['made', 'a', 'vow.'], ['a', 'vow.', 'For'], ['vow.', 'For', 'better'], ['For', 'better', 'or'], ['better', 'or', 'for'], ['or', 'for', 'worse.'], ['for', 'worse.', 'I'], ['worse.', 'I', "can't"], ['I', "can't", 'believe'], ["can't", 'believe', 'you'], ['believe', 'you', 'let'], ['you', 'let', 'me'], ['let', 'me', 'down']]

In [61]:


# res=three_pairwise(["A", "B", "C", "D", "E", "F", "G"])
res=zip
for key in docs.keys():
    mydoc = split_iter(docs[key])
    res=three_pairwise(mydoc) 

# print(res)
# for k in res:
    # print(k)
# res=more_itertools.pairwise()

('A', 'B', 'C')
('B', 'C', 'D')
('C', 'D', 'E')
('D', 'E', 'F')
('E', 'F', 'G')


In [249]:
s=["a","x"], ["b","x"], ["c","x"], ["d","x"]
a=['a', 'b', 'c', 'd']
# b_m=np.zeros((16,))
b_m=[]
for (i, j) in itertools.product(a, s):
    b_m.append(int(i in j))
    
print(b_m)
b_m=np.array(b_m)
np.reshape(b_m, (len(a),len(s)))
#   ax bx cx dx
# a 1  0  0  0
# b 0  1  0  0
# c 0  0  1  0
# d 0  0  0  1



[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]


array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

In [318]:
def ff(x):
    return x+1
a=itertools.starmap(ff, zip([1,2,3]))
for t in a:
    print(t)

2
3
4


In [33]:

a=np.array(["a", "b", "c", "d", "e", "f"])
b=np.array([["a", "b", "d", "d"], ['b','b,','b'], ['a','d','d','d'], ['d','d','d'], ['f','f','f']])
res_lst=[]
for c in b:
    my_a=np.zeros((len(a),))
    x = np.intersect1d(a, c, return_indices=True)
    # print(x[1]) # indices where similar elements exist in a
    np.put(my_a, x[1], [1])
    print(my_a)
    res_lst.append(my_a)
    # breaks
res_lst = np.vstack(res_lst)
res_lst

[1. 1. 0. 1. 0. 0.]
[0. 1. 0. 0. 0. 0.]
[1. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 0. 1.]


C:\Users\hasee\AppData\Local\Temp\ipykernel_5144\2502438174.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  b=np.array([["a", "b", "d", "d"], ['b','b,','b'], ['a','d','d','d'], ['d','d','d'], ['f','f','f']])


array([[1., 1., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [18]:
# import numpy as np
# a=np.array(["a", "b", "c", "d", "e", "f"])
# b=np.array([["a", "b", "d", "d"], ['b','b,','b'], ['a','d','d','d'], ['d','d','d'], ['f','f','f']])
# res_lst=[]
# seta=set(a)
# for c in b:
#     c=set(c)
#     my_a=np.zeros((len(a),))
#     res=list(seta.intersection(c))
#     indices=[np.argwhere(a == x) for x in res]
#     np.put(my_a, indices, [1])
#     print(my_a)
#     res_lst.append(my_a)
#     # breaks
# res_lst = np.vstack(res_lst)
# res_lst

['a', 'd', 'b']
[array([[0]], dtype=int64), array([[3]], dtype=int64), array([[1]], dtype=int64)]
end
[1. 1. 0. 1. 0. 0.]
['b']
[array([[1]], dtype=int64)]
end
[0. 1. 0. 0. 0. 0.]
['a', 'd']
[array([[0]], dtype=int64), array([[3]], dtype=int64)]
end
[1. 0. 0. 1. 0. 0.]
['d']
[array([[3]], dtype=int64)]
end
[0. 0. 0. 1. 0. 0.]
['f']
[array([[5]], dtype=int64)]
end
[0. 0. 0. 0. 0. 1.]


C:\Users\hasee\AppData\Local\Temp\ipykernel_17700\177946328.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  b=np.array([["a", "b", "d", "d"], ['b','b,','b'], ['a','d','d','d'], ['d','d','d'], ['f','f','f']])


array([[1., 1., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1.]])

### MinHashing implementation

In [88]:
import time
from itertools import repeat
def boolean_gen_subroutine(cart_prod_res_elem):
    # print("cart", cart_prod_list[0])
    # print("cccc", cart_prod_list[1])
    return int(cart_prod_res_elem[0] in cart_prod_res_elem[1]) ## return 1 if universe element exists in document

# def my_map_subroutine(single_u, doc):
#     set_uni=set(my_universe)
#     my_a=np.zeros((len(my_universe),))
#     res=list(set_uni.intersection(doc))
#     indices=[np.argwhere(my_universe == x) for x in res]
#     # x = np.intersect1d(my_universe, doc, return_indices=True)
#     # print(x[1]) # indices where similar elements exist in a
#     np.put(my_a, indices, [1])
#     return my_a

def boolean_gen(my_universe, my_docs, num_docs):
    # boolean_m=[]
    # for my_doc in doc_hashes1:
    # doc_hashes1=set(doc_hashes1) # Without this, the code is too slow to run!
    # return [u_elem in my_doc for my_doc in doc_hashes1] # [1 0 1 0 1 0]
    # #######################################################################
    b_m=[]
    # a = [[] for x in repeat(None, len(my_universe))]
    ######################################
    # my_universe = np.array(my_universe)
    # my_docs = np.array(my_docs)
    # print(my_universe.shape)
    # print(my_docs.shape)
    # print("hapes")
    
    # set_uni=set(my_universe)
    
    # for doc in my_docs:
    #     # print("DOC", doc)
    #     my_a=np.zeros((len(my_universe),))
    #     res=list(set_uni.intersection(doc))
    #     indices=[np.argwhere(my_universe == x) for x in res]
    #     # x = np.intersect1d(my_universe, doc, return_indices=True)
    #     # print(x[1]) # indices where similar elements exist in a
    #     np.put(my_a, indices, [1])
    #     b_m.append(my_a)
    
    # b_m = np.vstack(b_m)
    ########################################
    # then=time.time()
    # my_universe = np.array(my_universe)
    # map(my_map_subroutine, my_universe, docs)
    # # mapped_lst = [np.array(map(my_map_subroutine, my_universe, doc)) for doc in my_docs]
    # print("Elapsed time:", time.time()-then)
    # print(len(mapped_lst))
    # print(type(mapped_lst))
    # b_m=np.vstack(mapped_lst)
    ###########################################

    cart_prod_res=itertools.product(my_universe, my_docs) ## Cartesian product yields (u_elem1, [doc1_elem1, doc1_elem2, ...]), (u_elem2, [doc2_elem1, doc2_elem2 ... ]), ... Basically a nested for loop
    # cart_prod_list=list(zip(cart_prod_list))
    boolean_array=map(boolean_gen_subroutine, cart_prod_res)
    # boolean_array = itertools.starmap(boolean_gen_subroutine, cart_prod_list) # for each element in my_list we are checking whether u_elem exists in the doc if yes we place a 1 otherwise 0 to get boolean array
    # boolean_array=list(boolean_array) ## returns list of boolean array
    print("bool array type", type(boolean_array))
    # boolean_array=np.fromiter(boolean_array, dtype=np.int)
    # print("bool array size", boolean_array.shape)
    # boolean_array = np.reshape(boolean_array, (len(my_universe), num_docs))
    # print("bool array shape after reshaping", boolean_array.shape)
    # print("arra",boolean_array)
    # print("looping done")


    # print("np ara")
    # b_m=np.array(b_m)
    # print("uni", len(my_universe))
    print("np done")
    # b_m = np.reshape(b_m, (len(my_universe),num_docs)) ## we reshape to get a matrix from the list
    # print("ddddddddd")
    # print(len(my_universe))
    # print(num_docs)
    # print(b_m.shape)
    # return b_m
    return boolean_array


def generate_matrices(docs, shingle_size=3, num_hash_funcs=2):
    print("1")
    
    start=time.time()

    doc_hashes, num_docs = hashed_lst_shingles(docs)        
    doc_hashes1, doc_hashes2 = itertools.tee(doc_hashes, 2) ## create two iterators as one gets consumed
    
    # print()
    end=time.time()
    print("Elapsed 1", end-start)
    
    my_universe = list(itertools.chain(*doc_hashes2)) # flatten to get a single list
    my_universe = list(set(my_universe)) # set() to remove duplicates. This gives the universe.
    # print("my elems")
    # print(my_elements)
    print("3")
    ''' Iterate over documents (S1, S2, etc.).
        insert 1 if element from universe exists in the doc else 0
        finally, concatenate all lists to get matrix m
    '''
    # boolean_m = []
    # print("doc hashes")
    # print(doc_hashes)

    
    # boolean_m = itertools.starmap(boolean_gen, list(zip(my_universe)))
    print("num_docs", num_docs)
    m = boolean_gen(my_universe, list(doc_hashes1), num_docs) # Matrix m!
    print("m shape", m.shape)
    print("4")
    # m = np.array(boolean_m) # Matrix m!
   
    ''' Compute hashes to get matrix h_m'''
    rows = np.arange(len(my_universe))
    # h_m = np.full((num_hash_funcs, len(my_universe)), np.arange(len(my_universe)))
    # h_m[]
    print("5")
    hashes_lst = []
    
    for i in range(num_hash_funcs):
        # h_m[i, :] = mmh3.hash(h_m[i, :], seed=i)
        hashes_lst.append([mmh3.hash(x, seed=i) for x in rows])

    h_m = np.array(hashes_lst).T # Matrix h_m!
    # h_m=h_m.T
    print("end")

    return m.T, h_m ## strange that you transpose here? # m: (22, 3), h_m: (22, 2)

def min_hash_alg(m, h_m):
    sig_m = np.full((h_m.shape[1], m.shape[1]), np.inf) # Signature matrix. This stores the final result! ## (2 hash funcions, 3 documents)
    print("6")
    my_args = np.argwhere(m == 1) # 
    for args in my_args:
        row=args[0]
        column=args[1]
        for i in range(h_m.shape[1]): # 2
            if h_m[row][i] < sig_m[i, column]:
                sig_m[i, column] = h_m[row][i]
    
    return sig_m


In [101]:
lst=[[1,0,0], [0,0,0], [0,0,1], [1,1,1]]
sigm = np.full((3*2, ), np.inf)
for (i,j) in zip(lst, sigm):
    print(i)
    print(j)
    break

[1, 0, 0]
inf


In [54]:
np.stack([[1,2,3],[0,0,0]]).T

array([[1, 0],
       [2, 0],
       [3, 0]])

In [86]:
test_docs=list(zip(["You and me, we made a vow. For better or for worse. I can't believe you let me down",
"Time, space and state. Equal everything explanable.",
"You and me, we made a vow. For better or for worse. I can't believe you let me down"]))
m, h_m = generate_matrices(test_docs, shingle_size=3, num_hash_funcs=2)
print(m.shape)
print(h_m.shape)
sig_m = min_hash_alg(m, h_m)
sig_m
# m shape before transposing: (3, 22)
# m shape after transposing: (22, 3) 

1
Elapsed 1 0.0
3
num_docs 3
bool array type <class 'map'>
bool array size (66,)
bool array shape after reshaping (22, 3)
looping done
np done
m shape (3, 22)
4
5
end
(22, 3)
(22, 2)
6


C:\Users\hasee\AppData\Local\Temp\ipykernel_17700\1498093444.py:63: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  boolean_array=np.fromiter(boolean_array, dtype=np.int)


array([[-1.89304750e+09, -1.19706312e+09, -1.89304750e+09],
       [-7.78479335e+08, -2.07294148e+09, -7.78479335e+08]])

From results of sig matrix above, documents 1 and 3 are very similar (similar hashes)

In [25]:
''' Testing own implementation with input example from book'''
m = np.array([[1,0,0,1], [0,0,1,0], [0,1,0,1], [1,0,1,1],[0,0,1,0]])
h_m = np.array([[1,1],[2,4],[3,2],[4,0],[0,3]])
print("Signature matrix:")
sig_m = min_hash_alg(m, h_m)
sig_m

Signature matrix:
6


array([[1., 3., 0., 1.],
       [0., 2., 0., 0.]])

In [89]:
m, h_m = generate_matrices(docs, shingle_size=3, num_hash_funcs=100)
print("dd")
sig_m = min_hash_alg(m, h_m)
# sig_m

1
Elapsed 1 0.0
3
num_docs 1000
bool array type <class 'map'>
np done


AttributeError: 'map' object has no attribute 'shape'

In [9]:
h_m.shape

(379347, 100)

In [57]:
sig_m

array([[-2.14747874e+09, -2.14747580e+09, -2.14728504e+09,
        -2.14746990e+09, -2.14746630e+09, -2.14707323e+09,
        -2.14665401e+09, -2.14746630e+09],
       [-2.14746538e+09, -2.14724729e+09, -2.14715192e+09,
        -2.14748198e+09, -2.14744212e+09, -2.14738709e+09,
        -2.14715744e+09, -2.14743367e+09],
       [-2.14743896e+09, -2.14735663e+09, -2.14743896e+09,
        -2.14743896e+09, -2.14745845e+09, -2.14659046e+09,
        -2.14659046e+09, -2.14743182e+09],
       [-2.14746356e+09, -2.14741023e+09, -2.14742918e+09,
        -2.14746529e+09, -2.14744603e+09, -2.14743953e+09,
        -2.14743953e+09, -2.14744603e+09],
       [-2.14748073e+09, -2.14688307e+09, -2.14748073e+09,
        -2.14744967e+09, -2.14745262e+09, -2.14513463e+09,
        -2.14513463e+09, -2.14745262e+09],
       [-2.14747971e+09, -2.14737424e+09, -2.14746920e+09,
        -2.14745471e+09, -2.14747544e+09, -2.14745785e+09,
        -2.14745785e+09, -2.14743593e+09],
       [-2.14747919e+09, -2.147369

In [58]:
''' Jaccard similarity'''
def jaccard(s1, s2):
    return len(s1 & s2) / len(s1 | s2)

score = jaccard(set(sig_m[:, 5]), set(sig_m[:, 6])) # finding similarity b/w documents 5 and 6. remember00palm.txt and remembermeorholy00palm.txt

print(score)

0.6129032258064516


We see that docs 5 and 6 are quite similar (0.61).

### LSH implementation

In [66]:
''' Implementation of LSH, dividing signature matrix into b band with r rows each'''
def LSH(b, r):
    b=20
    r=5
    #b*r=num_hash_funcs

    sim_hashes=[]
    start=0
    for i in range(b):
        sim_hashes.append([listhash(col, seed=i) for col in sig_m[start:start+r,:].T])
        start=i+r

    return sim_hashes

''' Find candidate pairs by checking to see if the hashes match.
Then we check to see that the Jaccard similarity b/w each pair of docs is atleast t. If so, we consider it a candidate pair otherwise not '''
def get_cand_pairs(sim_hashes, t):
    cand_pairs=set()
    for L in sim_hashes:
        dups = collections.defaultdict(list)
        for i, e in enumerate(L):
            dups[e].append(i)
        for k, v in sorted(dups.items()):
            if len(v) >= 2:
                cand_pairs.add(tuple(v))
                # print(k, v)
    cand_pairs=list(cand_pairs)
    filtered_cand_pairs = [pair for pair in cand_pairs if (jaccard(set(sig_m[:, pair[0]]), set(sig_m[:, pair[1]])) > t)]
    return filtered_cand_pairs
    # return cand_pairs

In [67]:
b=20
r=5
sim_hashes = LSH(b=20, r=5)
pairs=get_cand_pairs(sim_hashes, t=(1/b)**(1/r))
pairs

[(5, 6)]

As can be seen using LSH, we find that documents 5 and 6 are similar.

### Development Junk

In [64]:
X = np.random.rand(10,5)

In [65]:
for col in X[0:3, :].T:
    print(col)

[0.76717558 0.16896812 0.60805319]
[0.10332414 0.71198539 0.31250171]
[0.08204225 0.43062834 0.52636287]
[0.94627444 0.85569538 0.23878093]
[0.5296346  0.48514638 0.3348344 ]
